# Class : Graphical Models - Markov Chains

---

## Before Class
In class today we will be implementing a Markov chain to process sentences
Prior to class, please do the following:
1. Review slides on Markov chains in detail
* Explore using the python dict() structure and how a dict() can contain nested dict() structures
* Again, review numpy.random.choice()

---
## Learning Objectives

1. Conceptually understand Markov Chains
* Implement a Markov Chain
* Generate data using Markov Chain

---
## Background

Recall from the lectures that Markov Chains represent a series of events following the Markov Property: future states are memory-less in that they depend only on the current state. This can be expanded to the idea of variable order Markov models where there is a variable-length memory (eg. 1st order Markov Model). Markov models consist of fully observable states. A common example of this is in predicting the weather: We can clearly see the current weather and would like to predict tomorrow's weather. As shown in the slides, this is also applicable to biology with one case being CpG islands. 

Our goal today will be to implement a Markov model built from words. For our example text, we will use the classic example of Dr. Seuss because of the repetitive nature of the text.

---
## Imports

In [4]:
import os
import numpy as np

---
## Train Markov model

For our initial implementation of the Markov Model, we will use the simple example of Dr. Seuss: "One fish two fish red fish blue fish."



In [5]:
def build_markov_model(markov_model, new_text):
    '''
    Function to build or add to a 1st order Markov model given a string of text
    We will store the markov model as a dictionary of dictionaries
    The key in the outer dictionary represents the current state
    and the inner dictionary represents the next state with their contents containing
    the transition probabilities.
    Note: This would be easier to read if we were to build a class representation
           of the model rather than a dictionary of dictionaries, but for simplicitiy
           our implementation will just use this structure.
    
    Args: 
        markov_model (dict of dicts): a dictionary of word:(next_word:frequency pairs)
        new_text (str): a string to build or add to the moarkov_model

    Returns:
        markov_model (dict of dicts): an updated markov_model
        
    Pseudocode:
        Add artificial states for start and end
        For each word in text:
            Increment markov_model[word][next_word]
        
    '''
    
    # Split up our sentence by spaces
    words = new_text.split()
    
    # Add our artifical start and end states to the sentence
    words.insert(0, "*S*")
    words.append("*E*")
    
    # Iterate over each word in the input
    for i in range(0, len(words)-1):
        
        # Here we do a lot of checking to properly initialize the dictionaries
        # But the main goal is to increment a counter if we have seen word[i+1]
        # after word[i]
        if words[i] in markov_model:                     # If we have already seen this word
            if words[i+1] in markov_model[words[i]]:     # and if we have already seen the next word
                markov_model[words[i]][words[i+1]] += 1  # increment the word counter
            else:
                markov_model[words[i]][words[i+1]] = 1   # If we haven't seen the next word, create it
        else:
            markov_model[words[i]] = {}                  # If we haven't see the word then create it
            markov_model[words[i]][words[i+1]] = 1
    
    return markov_model
    


In [6]:
markov_model = dict()
text = "one fish two fish red fish blue fish"
markov_model = build_markov_model(markov_model, text)
print (markov_model)

{'*S*': {'one': 1}, 'one': {'fish': 1}, 'fish': {'two': 1, 'red': 1, 'blue': 1, '*E*': 1}, 'two': {'fish': 1}, 'red': {'fish': 1}, 'blue': {'fish': 1}}


## Generate text from Markov Model

Markov models are "generative models". That is, the probability states in the model can be used to generate output following the conditional probabilities in the model.

We will now generate a sequence of text from the Markov model:

In [7]:
def get_next_word(current_word, markov_model):
    '''
    Function to randomly move a valid next state given a markov model
    and a current state (word)
    
    Args: 
        current_word (str): a word that exists in our model
        markov_model (dict of dicts): a dictionary of word:(next_word:frequency pairs)

    Returns:
        next_word (str): a randomly selected next word based on transition probabilies
        
    Pseudocode:
        Calculate transition probilities for all next states from a given state (counts/sum)
        Randomly draw from these to generate the next state
        
    '''
    # Get all of our possible next states
    next_words = list(markov_model[current_word].keys())
    
    # Calculate the probabilities to move to those based on word counts
    next_words_frequencies = list(markov_model[current_word].values())
    next_words_probabilities = [x / sum(next_words_frequencies) for x in next_words_frequencies]

    # Randomly move to the next state
    next_state = np.random.choice( next_words, 1, p=next_words_probabilities)

    # Return next state
    return next_state[0]

    
def generate_random_text(markov_model):
    '''
    Function to generate text given a markov model
    
    Args: 
        markov_model (dict of dicts): a dictionary of word:(next_word:frequency pairs)

    Returns:
        sentence (str): a randomly generated sequence given the model
        
    Pseudocode:
        Initialize sentence at start state
        Until End State:
            append get_next_word(current_word, markov_model)
        Return sentence
        
    '''
    # We must start at the initial state of the model
    current_word = "*S*"
    
    # Keeping track of the sentence as a list (ignoring the start state)
    sentence = []

    # Until the model generates an end state, keep adding random words
    while current_word != "*E*":
        current_word = get_next_word(current_word, markov_model)
        
        # Don't append the end state to our output
        if current_word != "*E*":
            sentence.append(current_word)
    
    # Return the words with spaces between them
    return ' '.join(sentence)

In [8]:
# Print a random sentence from our markov chain:
print (generate_random_text(markov_model))

one fish two fish red fish


In [9]:
# Now just add some more training data to the markov model
markov_model = build_markov_model(markov_model, "black fish blue fish old fish new fish")
markov_model = build_markov_model(markov_model, "this one has a little car")
markov_model = build_markov_model(markov_model, "this one has a little star")
markov_model = build_markov_model(markov_model, "say what a lot of fish there are")
markov_model = build_markov_model(markov_model, "yes some are red and some are blue")
markov_model = build_markov_model(markov_model, "some are old and some are new")

print(markov_model)

{'*S*': {'one': 1, 'black': 1, 'this': 2, 'say': 1, 'yes': 1, 'some': 1}, 'one': {'fish': 1, 'has': 2}, 'fish': {'two': 1, 'red': 1, 'blue': 2, '*E*': 2, 'old': 1, 'new': 1, 'there': 1}, 'two': {'fish': 1}, 'red': {'fish': 1, 'and': 1}, 'blue': {'fish': 2, '*E*': 1}, 'black': {'fish': 1}, 'old': {'fish': 1, 'and': 1}, 'new': {'fish': 1, '*E*': 1}, 'this': {'one': 2}, 'has': {'a': 2}, 'a': {'little': 2, 'lot': 1}, 'little': {'car': 1, 'star': 1}, 'car': {'*E*': 1}, 'star': {'*E*': 1}, 'say': {'what': 1}, 'what': {'a': 1}, 'lot': {'of': 1}, 'of': {'fish': 1}, 'there': {'are': 1}, 'are': {'*E*': 1, 'red': 1, 'blue': 1, 'old': 1, 'new': 1}, 'yes': {'some': 1}, 'some': {'are': 4}, 'and': {'some': 2}}


In [10]:
# And print a more complex sentence
print (generate_random_text(markov_model))

some are red and some are old fish blue fish new fish blue
